In [ ]:
"""
LANGGRAPH PARALLEL WORKFLOWS - Product Analysis System

A complete example showing how to:
1. Split work into multiple parallel branches
2. Merge results back together
3. Generate final comprehensive report

Use Case: Analyze a product from multiple angles simultaneously:
- Marketing Analysis (Target audience, positioning, messaging)
- Technical Analysis (Features, architecture, tech stack)
- Financial Analysis (Pricing, revenue potential, ROI)
- Competitive Analysis (Market position, competitors, differentiation)

SETUP:
pip install langgraph openai python-dotenv

Create .env:
OPENAI_API_KEY=sk-proj-your-key
"""

from langgraph.graph import StateGraph, START, END
from typing import TypedDict, Annotated
from openai import OpenAI
import os
import json
from operator import add
from dotenv import load_dotenv

load_dotenv()

# ============================================================================
# SETUP
# ============================================================================
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    print("❌ ERROR: Set OPENAI_API_KEY environment variable")
    exit(1)

client = OpenAI(api_key=OPENAI_API_KEY)

# ============================================================================
# STATE DEFINITION
# ============================================================================
class AnalysisState(TypedDict):
    """
    State for parallel product analysis workflow.
    
    Fields:
    - product_info: Contains product_name and description (read-only for branches)
    - parallel_analyses: Results from parallel branches
    - marketing_analysis: Marketing perspective
    - technical_analysis: Technical perspective
    - financial_analysis: Financial perspective
    - competitive_analysis: Competitive perspective
    - final_report: Merged comprehensive report
    - execution_log: Track what happened
    """
    product_info: dict  # Read-only: {"name": str, "description": str}
    parallel_analyses: Annotated[list, add]  # Accumulate parallel results
    marketing_analysis: str
    technical_analysis: str
    financial_analysis: str
    competitive_analysis: str
    final_report: str
    execution_log: Annotated[list, add]  # Track execution

# ============================================================================
# UTILITY FUNCTION: Call OpenAI
# ============================================================================
def call_openai(system_prompt: str, user_message: str, max_tokens: int = 300) -> str:
    """
    Helper to call OpenAI API consistently.
    
    Args:
        system_prompt: System message
        user_message: User question
        max_tokens: Response length
        
    Returns:
        Response text
    """
    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            max_tokens=max_tokens,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_message}
            ]
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"Error: {str(e)[:100]}"

# ============================================================================
# NODE 1: INPUT VALIDATION & PREPARATION
# ============================================================================
def validate_input(state: AnalysisState) -> AnalysisState:
    """Validate and prepare input for analysis"""
    print("\n🔍 NODE 1: Validating Input...")
    
    product_info = state.get("product_info", {})
    product_name = product_info.get("name", "Unknown Product")
    product_description = product_info.get("description", "No description provided")
    
    state["execution_log"].append("✓ Input validated")
    print(f"   Product: {product_name}")
    print(f"   Description: {product_description[:50]}...")
    
    return state

# ============================================================================
# PARALLEL BRANCH 1: MARKETING ANALYSIS
# ============================================================================
def marketing_analysis(state: AnalysisState) -> AnalysisState:
    """Analyze product from marketing perspective"""
    print("\n📊 BRANCH 1: Marketing Analysis (PARALLEL)")
    
    product_info = state.get("product_info", {})
    product_name = product_info.get("name", "Unknown")
    product_description = product_info.get("description", "No description")
    
    prompt = f"""Analyze this product from a MARKETING perspective:

Product: {product_name}
Description: {product_description}

Provide analysis on:
1. Target Audience (who should use this?)
2. Unique Value Proposition (why choose this?)
3. Marketing Messaging (key benefits)
4. Pricing Strategy (recommended pricing)

Keep each point to 1-2 sentences."""
    
    analysis = call_openai(
        system_prompt="You are an expert marketing strategist analyzing products.",
        user_message=prompt,
        max_tokens=300
    )
    
    state["marketing_analysis"] = analysis
    state["parallel_analyses"].append("✓ Marketing analysis completed")
    state["execution_log"].append("✓ Marketing analysis")
    print(f"   Result: {analysis[:80]}...")
    
    return state

# ============================================================================
# PARALLEL BRANCH 2: TECHNICAL ANALYSIS
# ============================================================================
def technical_analysis(state: AnalysisState) -> AnalysisState:
    """Analyze product from technical perspective"""
    print("\n⚙️  BRANCH 2: Technical Analysis (PARALLEL)")
    
    product_info = state.get("product_info", {})
    product_name = product_info.get("name", "Unknown")
    product_description = product_info.get("description", "No description")
    
    prompt = f"""Analyze this product from a TECHNICAL perspective:

Product: {product_name}
Description: {product_description}

Provide analysis on:
1. Core Features (main functionalities)
2. Technology Stack (suggested technologies)
3. Architecture Approach (how to build this)
4. Scalability Considerations (growth handling)

Keep each point to 1-2 sentences."""
    
    analysis = call_openai(
        system_prompt="You are a senior technical architect evaluating products.",
        user_message=prompt,
        max_tokens=300
    )
    
    state["technical_analysis"] = analysis
    state["parallel_analyses"].append("✓ Technical analysis completed")
    state["execution_log"].append("✓ Technical analysis")
    print(f"   Result: {analysis[:80]}...")
    
    return state

# ============================================================================
# PARALLEL BRANCH 3: FINANCIAL ANALYSIS
# ============================================================================
def financial_analysis(state: AnalysisState) -> AnalysisState:
    """Analyze product from financial perspective"""
    print("\n💰 BRANCH 3: Financial Analysis (PARALLEL)")
    
    product_info = state.get("product_info", {})
    product_name = product_info.get("name", "Unknown")
    product_description = product_info.get("description", "No description")
    
    prompt = f"""Analyze this product from a FINANCIAL perspective:

Product: {product_name}
Description: {product_description}

Provide analysis on:
1. Revenue Potential (income opportunities)
2. Cost Structure (operational costs)
3. Profitability Timeline (when profitable?)
4. Funding Needs (capital requirements)

Keep each point to 1-2 sentences."""
    
    analysis = call_openai(
        system_prompt="You are a financial analyst evaluating business opportunities.",
        user_message=prompt,
        max_tokens=300
    )
    
    state["financial_analysis"] = analysis
    state["parallel_analyses"].append("✓ Financial analysis completed")
    state["execution_log"].append("✓ Financial analysis")
    print(f"   Result: {analysis[:80]}...")
    
    return state

# ============================================================================
# PARALLEL BRANCH 4: COMPETITIVE ANALYSIS
# ============================================================================
def competitive_analysis(state: AnalysisState) -> AnalysisState:
    """Analyze product from competitive perspective"""
    print("\n🏆 BRANCH 4: Competitive Analysis (PARALLEL)")
    
    product_info = state.get("product_info", {})
    product_name = product_info.get("name", "Unknown")
    product_description = product_info.get("description", "No description")
    
    prompt = f"""Analyze this product from a COMPETITIVE perspective:

Product: {product_name}
Description: {product_description}

Provide analysis on:
1. Market Position (where in the market?)
2. Main Competitors (who else offers this?)
3. Competitive Advantages (what makes it better?)
4. Market Entry Strategy (how to win market share?)

Keep each point to 1-2 sentences."""
    
    analysis = call_openai(
        system_prompt="You are a competitive strategist analyzing market dynamics.",
        user_message=prompt,
        max_tokens=300
    )
    
    state["competitive_analysis"] = analysis
    state["parallel_analyses"].append("✓ Competitive analysis completed")
    state["execution_log"].append("✓ Competitive analysis")
    print(f"   Result: {analysis[:80]}...")
    
    return state

# ============================================================================
# MERGE NODE: COMBINE PARALLEL RESULTS
# ============================================================================
def merge_analyses(state: AnalysisState) -> AnalysisState:
    """
    Merge all parallel analyses into one state.
    This node waits for all parallel branches to complete.
    """
    print("\n🔄 MERGE: Combining Parallel Results...")
    
    state["execution_log"].append("✓ All parallel branches completed")
    state["execution_log"].append("✓ Merging results...")
    
    print(f"   Parallel results: {state['parallel_analyses']}")
    print(f"   All analyses ready for final report")
    
    return state

# ============================================================================
# FINAL NODE: GENERATE COMPREHENSIVE REPORT
# ============================================================================
def generate_final_report(state: AnalysisState) -> AnalysisState:
    """Generate comprehensive report from all analyses"""
    print("\n📋 FINAL NODE: Generating Comprehensive Report...")
    
    prompt = f"""Create a COMPREHENSIVE PRODUCT ANALYSIS REPORT:

PRODUCT: {state['product_name']}

MARKETING PERSPECTIVE:
{state['marketing_analysis']}

TECHNICAL PERSPECTIVE:
{state['technical_analysis']}

FINANCIAL PERSPECTIVE:
{state['financial_analysis']}

COMPETITIVE PERSPECTIVE:
{state['competitive_analysis']}

Now synthesize this into a cohesive executive summary that:
1. Summarizes the overall opportunity
2. Highlights key strengths from each perspective
3. Identifies critical risks
4. Recommends next steps
5. Provides overall viability score (1-10)

Format as a professional executive summary."""
    
    report = call_openai(
        system_prompt="You are a senior business consultant synthesizing complex analysis.",
        user_message=prompt,
        max_tokens=500
    )
    
    state["final_report"] = report
    state["execution_log"].append("✓ Final report generated")
    print(f"   Report ready!")
    
    return state

# ============================================================================
# BUILD THE PARALLEL WORKFLOW GRAPH
# ============================================================================
def build_graph():
    """
    Build the parallel workflow graph.
    
    Execution Flow:
    
    START
      ↓
    validate_input
      ↓
    ┌────────────────────────────────────┐
    │  PARALLEL BRANCHES                 │
    │  (All execute simultaneously)       │
    │  ├─ marketing_analysis             │
    │  ├─ technical_analysis             │
    │  ├─ financial_analysis             │
    │  └─ competitive_analysis           │
    └────────────────────────────────────┘
      ↓
    merge_analyses
      ↓
    generate_final_report
      ↓
    END
    """
    
    workflow = StateGraph(AnalysisState)
    
    # Add all nodes
    workflow.add_node("validate", validate_input)
    workflow.add_node("marketing", marketing_analysis)
    workflow.add_node("technical", technical_analysis)
    workflow.add_node("financial", financial_analysis)
    workflow.add_node("competitive", competitive_analysis)
    workflow.add_node("merge", merge_analyses)
    workflow.add_node("report", generate_final_report)
    
    # Sequential edges
    workflow.add_edge(START, "validate")
    
    # Parallel edges: From validate to all 4 analysis branches
    workflow.add_edge("validate", "marketing")
    workflow.add_edge("validate", "technical")
    workflow.add_edge("validate", "financial")
    workflow.add_edge("validate", "competitive")
    
    # All branches converge to merge
    workflow.add_edge("marketing", "merge")
    workflow.add_edge("technical", "merge")
    workflow.add_edge("financial", "merge")
    workflow.add_edge("competitive", "merge")
    
    # Then to final report
    workflow.add_edge("merge", "report")
    workflow.add_edge("report", END)

    
    

    
    return workflow.compile()

# ============================================================================
# RUN THE WORKFLOW
# ============================================================================
def analyze_product(product_name: str, product_description: str):
    """
    Run complete parallel analysis workflow.
    
    Args:
        product_name: Name of product to analyze
        product_description: Description of what it does
    """
    
    initial_state = AnalysisState(
        product_info={"name": product_name, "description": product_description},
        parallel_analyses=[],
        marketing_analysis="",
        technical_analysis="",
        financial_analysis="",
        competitive_analysis="",
        final_report="",
        execution_log=[]
    )
    
    print("\n" + "="*80)
    print(f"🚀 PARALLEL WORKFLOW: Product Analysis System")
    print("="*80)
    print(f"Product: {product_name}")
    print(f"Description: {product_description}")
    print("="*80)
    
    # Build and invoke graph
    graph = build_graph()
    final_state = graph.invoke(initial_state)
    
    # Display results
    print("\n" + "="*80)
    print("📊 EXECUTION SUMMARY")
    print("="*80)
    for log in final_state["execution_log"]:
        print(f"  {log}")
    
    print("\n" + "="*80)
    print("📈 MARKETING ANALYSIS")
    print("="*80)
    print(final_state["marketing_analysis"])
    
    print("\n" + "="*80)
    print("⚙️  TECHNICAL ANALYSIS")
    print("="*80)
    print(final_state["technical_analysis"])
    
    print("\n" + "="*80)
    print("💰 FINANCIAL ANALYSIS")
    print("="*80)
    print(final_state["financial_analysis"])
    
    print("\n" + "="*80)
    print("🏆 COMPETITIVE ANALYSIS")
    print("="*80)
    print(final_state["competitive_analysis"])
    
    print("\n" + "="*80)
    print("📋 COMPREHENSIVE EXECUTIVE REPORT")
    print("="*80)
    print(final_state["final_report"])
    
    print("\n" + "="*80)
    print("✅ WORKFLOW COMPLETED")
    print("="*80 + "\n")
    
    return final_state

# ============================================================================
# TEST CASES
# ============================================================================
if __name__ == "__main__":
 
    
    print("\n╔" + "="*78 + "╗")
    print("║" + " "*15 + "LANGGRAPH PARALLEL WORKFLOWS - PRODUCT ANALYSIS" + " "*20 + "║")
    print("╚" + "="*78 + "╝\n")
    
    # Test Case 1: AI Writing Assistant
    print("\n🔵 TEST CASE 1: AI Writing Assistant Tool\n")
    result1 = analyze_product(
        product_name="AI Writing Assistant",
        product_description="An AI-powered tool that helps users write better emails, documents, and social media posts with real-time suggestions and grammar checking."
    )
    
    # Test Case 2: Fitness Tracking App
    print("\n🟢 TEST CASE 2: Fitness Tracking Mobile App\n")
    result2 = analyze_product(
        product_name="FitTrack Pro",
        product_description="A mobile app that tracks workouts, calculates calories, provides personalized training plans, and syncs with wearable devices."
    )
    
    # Test Case 3: Project Management Tool
    print("\n🟡 TEST CASE 3: Project Management SaaS\n")
    result3 = analyze_product(
        product_name="TaskFlow",
        product_description="A web-based project management platform for teams to collaborate, track tasks, manage deadlines, and visualize project progress with Gantt charts and Kanban boards."
    )
    
    print("\n" + "="*80)
    print("✨ ALL PARALLEL WORKFLOWS COMPLETED SUCCESSFULLY!")
    print("="*80 + "\n")

SyntaxError: invalid syntax (464152074.py, line 460)